In [115]:
import pandas as pd
ss_seasonal_storage  = pd.read_csv('../networks/OpenEntrance/SS_SeasonalStorage.csv', sep=';') 
iamc_annual_data = pd.read_csv('../networks/OpenEntrance/IAMC_annual_data.csv')
in_interconnections = pd.read_csv('../networks/OpenEntrance/IN_Interconnections.csv')
res_renewable_units = pd.read_csv('../networks/OpenEntrance/RES_RenewableUnits.csv')
sts_short_term_storage = pd.read_csv('../networks/OpenEntrance/STS_ShortTermStorage.csv', sep=';')
tu_thermal_units = pd.read_csv('../networks/OpenEntrance/TU_ThermalUnits.csv')
zp_zone_partition = pd.read_csv('../networks/OpenEntrance/ZP_ZonePartition.csv')
zv_zone_values = pd.read_csv('../networks/OpenEntrance/ZV_ZoneValues.csv')





# Define the NUTS-2 to region names mapping


In [116]:
nuts_to_region = {
    "ES11": "Galicia",
    "ES12": "Principado de Asturias",
    "ES13": "Cantabria",
    "ES21": "País Vasco",
    "ES22": "Comunidad Foral de Navarra",
    "ES23": "La Rioja",
    "ES24": "Aragón",
    "ES30": "Comunidad de Madrid",
    "ES41": "Castilla y León",
    "ES42": "Castilla-La Mancha",
    "ES43": "Extremadura",
    "ES51": "Cataluña",
    "ES52": "Comunidad Valenciana",
    "ES53": "Islas Baleares",
    "ES61": "Andalucía",
    "ES62": "Región de Murcia",
    "ES63": "Ciudad Autónoma de Ceuta",
    "ES64": "Ciudad Autónoma de Melilla",
    "ES70": "Canarias"
}


# Split the combined columns in the RES and TU files


In [117]:

res_renewable_units_split = res_renewable_units['Name;Zone;NumberUnits;MaxPower;MinPower;MaxPowerProfile;Energy;Kappa;Capacity;MaxAddedCapacity;MaxRetCapacity;InvestmentCost'].str.split(';', expand=True)
res_renewable_units_split.columns = ['Name', 'Zone', 'NumberUnits', 'MaxPower', 'MinPower', 'MaxPowerProfile', 'Energy', 'Kappa', 'Capacity', 'MaxAddedCapacity', 'MaxRetCapacity', 'InvestmentCost']

tu_thermal_units_split = tu_thermal_units['Zone;Name;NumberUnits;MaxPower;VariableCost;FixedCost;InvestmentCost;MaxAddedCapacity;MaxRetCapacity'].str.split(';', expand=True)
tu_thermal_units_split.columns = ['Zone', 'Name', 'NumberUnits', 'MaxPower', 'VariableCost', 'FixedCost', 'InvestmentCost', 'MaxAddedCapacity', 'MaxRetCapacity']



In [118]:
# Convert it in numerical data
res_renewable_units_split['MaxPower'] = pd.to_numeric(res_renewable_units_split['MaxPower'], errors='coerce')
res_renewable_units_split['NumberUnits'] = pd.to_numeric(res_renewable_units_split['NumberUnits'], errors='coerce')
tu_thermal_units_split['MaxPower'] = pd.to_numeric(tu_thermal_units_split['MaxPower'], errors='coerce')
tu_thermal_units_split['NumberUnits'] = pd.to_numeric(tu_thermal_units_split['NumberUnits'], errors='coerce')


In [119]:
# Combine dataframes
generators = pd.concat([res_renewable_units_split[['Name', 'Zone', 'NumberUnits', 'MaxPower']],
                         tu_thermal_units_split[['Name', 'Zone', 'NumberUnits', 'MaxPower']]], ignore_index=True)

# Map the zones and the regions
generators['Region'] = generators['Zone'].map(nuts_to_region)

# Group by region and generator type

In [120]:
# Agrupar por región y tipo de generador
grouped_generators = generators.groupby(['Region', 'Name']).agg({'NumberUnits': 'sum', 'MaxPower': 'sum'}).reset_index()

# Mostrar el DataFrame agrupado
print(grouped_generators)

# Guardar el DataFrame agrupado en un archivo CSV
grouped_generators.to_csv('Grouped_Generators_by_Region_and_Type.csv', index=False)

                    Region     Name  NumberUnits  MaxPower
0                Andalucía     CCGT            1   1738.70
1                Andalucía    CCGT2            1    482.50
2                Andalucía    CCGT3            1   3977.80
3                Andalucía  biomass            1     20.00
4                Andalucía     coal            1    543.02
..                     ...      ...          ...       ...
60  Principado de Asturias    solar            1    250.14
61        Región de Murcia     CCGT            1   3425.30
62        Región de Murcia     coal            1   1068.52
63        Región de Murcia   onwind            1    648.00
64        Región de Murcia    solar            1    557.19

[65 rows x 4 columns]


# What RES + ROR is in that region

In [121]:
# From the res_renewable_units_split dataframe, filter the data which starts with ES from the Zone column
res_renewable_units_split = res_renewable_units_split[res_renewable_units_split['Zone'].str.startswith('ES')]

# Map zones to region names
res_renewable_units_split['Region'] = res_renewable_units_split['Zone'].map(nuts_to_region)


In [122]:
# Group by region and type of generator
grouped_res_renewable_units_split = res_renewable_units_split.groupby(['Region', 'Name']).agg({'NumberUnits': 'sum', 'MaxPower': 'sum'}).reset_index()

# Display the grouped DataFrame
print(grouped_res_renewable_units_split)

# Save the grouped DataFrame to a CSV file
grouped_res_renewable_units_split.to_csv('Grouped_RES_RenewableUnits_by_Region_and_Type.csv', index=False)

                        Region    Name  NumberUnits  MaxPower
0                    Andalucía  onwind            1   2660.22
1                    Andalucía   solar            1   1779.21
2                       Aragón  onwind            1    945.95
3                       Aragón     ror            1     36.65
4                       Aragón   solar            1    480.17
5                    Cantabria  onwind            1   1386.04
6                    Cantabria   solar            1    379.39
7              Castilla y León  onwind            1   1162.47
8              Castilla y León   solar            1    397.93
9           Castilla-La Mancha  onwind            1   4457.61
10          Castilla-La Mancha   solar            1   2463.28
11                    Cataluña  onwind            1    363.48
12                    Cataluña     ror            1    110.36
13                    Cataluña   solar            1    287.89
14  Comunidad Foral de Navarra  onwind            1   2389.28
15  Comu

# What pumped storage is in the region

In [123]:
# Convert to numeric data
sts_short_term_storage['MaxPower'] = pd.to_numeric(sts_short_term_storage['MaxPower'], errors='coerce')
sts_short_term_storage['NumberUnits'] = pd.to_numeric(sts_short_term_storage['NumberUnits'], errors='coerce')




In [124]:
# Filter the data for pumped storage
pumped_storage = sts_short_term_storage[sts_short_term_storage['AddPumpedStorage'].notnull()]

# Map zones to region names
pumped_storage['Region'] = pumped_storage['Zone'].map(nuts_to_region)


In [125]:
# Group by region and type of pumped storage
grouped_pumped_storage = pumped_storage.groupby(['Region', 'Name']).agg({'NumberUnits': 'sum', 'MaxPower': 'sum'}).reset_index()

# Display the grouped DataFrame
print(grouped_pumped_storage)

                        Region            Name  NumberUnits     MaxPower
0                    Andalucía     Lithium-Ion            1  2647.381159
1                    Andalucía  Pumped Storage            1  2385.361183
2                       Aragón     Lithium-Ion            1   413.231542
3                       Aragón  Pumped Storage            1   372.332665
4                    Cantabria     Lithium-Ion            1   183.207607
5                    Cantabria  Pumped Storage            1   165.074950
6              Castilla y León     Lithium-Ion            1   760.694778
7              Castilla y León  Pumped Storage            1   685.406327
8           Castilla-La Mancha     Lithium-Ion            1   639.965358
9           Castilla-La Mancha  Pumped Storage            1   576.625892
10                    Cataluña     Lithium-Ion            1  2399.724452
11                    Cataluña  Pumped Storage            1  2162.215871
12  Comunidad Foral de Navarra     Lithium-Ion     

In [126]:
# Save the grouped DataFrame to a CSV file
grouped_pumped_storage.to_csv('Grouped_Pumped_Storage_by_Region_and_Type.csv', index=False)

# Seasonal Storage and how much

In [127]:
# Convert to numeric data
ss_seasonal_storage['MaxPower'] = pd.to_numeric(ss_seasonal_storage['MaxPower'], errors='coerce')
ss_seasonal_storage['NumberUnits'] = pd.to_numeric(ss_seasonal_storage['NumberUnits'], errors='coerce')

In [129]:
# Filter the data for seasonal storage which is the MaxPower.
seasonal_storage = ss_seasonal_storage[ss_seasonal_storage['MaxPower'].notnull()]
seasonal_storage
# Filter the data for pumped storage
seasonal_storage = ss_seasonal_storage[ss_seasonal_storage['AddPumpedStorage'].notnull()]

# Map zones to region names
ss_seasonal_storage['Region'] = ss_seasonal_storage['Zone'].map(nuts_to_region)
# Group by region and type of pumped storage
grouped_ss_seasonal_storage = ss_seasonal_storage.groupby(['Region', 'Name']).agg({'NumberUnits': 'sum', 'MaxPower': 'sum'}).reset_index()

# Display the grouped DataFrame
print(grouped_ss_seasonal_storage )


     Region       Name  NumberUnits  MaxPower
0  Cataluña  Reservoir            1    7490.0


# Load proportion

In [130]:
# Filter for total load values
total_load = zv_zone_values[zv_zone_values['Type'] == 'Total']

# Extract Spain's total load value
spain_load = total_load[total_load['Zone'] == 'Spain']['value'].values[0]

In [131]:
# Calculate the total load for all zones
overall_load = total_load['value'].sum()

# Calculate the proportion of Spain's load to the overall load
spain_load_proportion = spain_load / overall_load

# Print the results
print(f"Spain's Load: {spain_load}")
print(f"Overall Load: {overall_load}")
print(f"Proportion of Spain's Load: {spain_load_proportion}")

Spain's Load: 357831438.52732056
Overall Load: 2690472938.6805863
Proportion of Spain's Load: 0.13299945648321662


# Regions connected and transported back

In [133]:
# Filter interconnections involving Spain
spain_interconnections = in_interconnections[in_interconnections['Name'].str.contains('Spain')]

# Extract relevant details about the connections
spain_connections = spain_interconnections[['Name', 'MaxPowerFlow', 'MinPowerFlow']]

# Display the details
print(spain_connections)

              Name  MaxPowerFlow  MinPowerFlow
0   Portugal>Spain        4500.0       -4100.0
1  Spain>BigFrance        5000.0       -5000.0
